<a href="https://colab.research.google.com/github/thegreatwarlo/BeerPersonalization/blob/master/Data_parsing_and_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's how you can load file from your google Drive. (the path might change)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

TIMEOUT: ignored

In [0]:
import gzip

with gzip.open('/content/gdrive/My Drive/Columbia/Personalization/Final Project/Beeradvocate.txt.gz', 'r') as f:
  rb_file = f.readlines()

In [3]:
#Use this if your file is in txt format

with open('your path', 'r') as f:
  rb_file = f.readlines()

FileNotFoundError: ignored

In [0]:
# python2
import pandas as pd
import numpy as np
import os

In [0]:
data = []
row_out = []

for i in rb_file:
    row = i.decode('utf-8', errors = 'ignore')
    #print(row)
    if row == '\n':
      data.append(row_out)
      row_out = []
      continue
    row = row.split(":", 1)[1].rstrip()
    row_out.append(row)

        
        

In [26]:
data = pd.DataFrame(data)

data.columns = ['beer_name', 'beer_beerId', 'beer_brewer', 'beer_ABV', 'beer_style', 
                'review_appearance', 'review_aroma', 'review_palate', 'review_taste', 
                'review_overall', 'review_time', 'review_profileName', 'review_text']

# keep 3 columns: user name, beer name, overall score
data2 = data[['beer_name', 'review_profileName', 'review_overall','review_time']]
#print(data2.shape)

# remove NA
data2 = data2[pd.notnull(data2.beer_name)]
data2 = data2[pd.notnull(data2.review_profileName)]
data2 = data2[pd.notnull(data2.review_overall)]
print(data2.shape)
#data2.head(10)

(1586614, 4)


In [27]:
# keep top 2000/33382 frequent users
user = data2.review_profileName.value_counts()
user_list = user.keys()[:2000].tolist()

# keep top 200/56855 most reviewed beer
beer = data2.beer_name.value_counts()[:200]
beer_list = beer.keys()[:200].tolist()

# keep (beer&user) pair in (user_list) and (beer_list)
subdata = data2[data2.beer_name.isin(beer_list)]
subdata = subdata[subdata.review_profileName.isin(user_list)]

# sort by user names
subdata = subdata.sort_values(by=['review_profileName','beer_name','review_time'])

print(subdata.shape)
subdata.head(10)

(177350, 4)


,beer_name,review_profileName,review_overall,review_time
203255,1554 Enlightened Black Ale,,5,1230572555
1171411,60 Minute IPA,,5,1092634453
1170196,60 Minute IPA,,4.5,1231377089
1175028,90 Minute IPA,,5,1120085235
179764,AleSmith Speedway Stout,,4.5,1104186971
1320696,Anchor Porter,,5,1085582845
1322585,Anchor Steam Beer,,3.5,1161395034
1190684,ApriHop,,4,1238969849
869290,Arrogant Bastard Ale,,4.5,1082163606
868115,Arrogant Bastard Ale,,4.5,1215045531


Only keeping latest reviews by user in case they rated the same beer multipe times

In [0]:
#subdata.groupby(['beer_name', 'review_profileName'])['review_time'].max()

data_clean = subdata.sort_values(by='review_time')

data_clean.drop_duplicates(['beer_name', 'review_profileName'], keep = 'last',
                          inplace = True)


In [31]:
data_clean.head(10)
#data_clean.shape

,beer_name,review_profileName,review_overall,review_time
282223,Samuel Adams Boston Lager,Jason,4,1000083887
292025,Samuel Adams Octoberfest,AaronRed,4,1000135216
518392,Samuel Smith's Oatmeal Stout,Jason,4,1000336190
518391,Samuel Smith's Oatmeal Stout,guinness33,5,1000464943
1440424,Black Butte Porter,Tim,5,1000674768
1339361,Alpha King Pale Ale,John,5,1000733852
1148316,Fuller's ESB,John,5,1000804529
1541504,Golden Monkey,John,5,1000809985
344591,Orval Trappist Ale,AaronRed,5,1000832810
275832,Hoegaarden Original White Ale,AaronRed,4,1000833130
